In [1]:
import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm.notebook import tqdm

In [2]:
ee.Initialize()

In [3]:
start_date = "2000-01-01"
end_date = "2023-12-31"

In [4]:
spain_map = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(
    ee.Filter.eq("ADM0_NAME", "Spain")
)

In [5]:
spain_provinces = gpd.read_file("../data/maps/spain_map.geojson")

In [6]:
spain_provinces = spain_provinces[
    spain_provinces["community"]
    != "Gibraltar. Territorio no asociado a ninguna autonomía"
]

In [7]:
terra_climate_dataset = (
    ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
    .filterBounds(spain_map)
    .filterDate(start_date, end_date)
)

In [8]:
def get_monthly_data(feature):
    province_name = feature["properties"]["province"]
    province_geometry = feature["geometry"]

    def spatial_aggregation(image):
        mean_precipitation_dict = image.select("pr").reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=province_geometry,
            scale=4638.3,
        )

        mean_pet_dict = image.select("pet").reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=province_geometry,
            scale=4638.3,
        )

        date = image.date().format("YYYY-MM-dd")

        feature = ee.Feature(
            None,
            {
                "province": province_name,
                "date": date,
                "avg_precipitation": mean_precipitation_dict.get("pr"),
                "avg_pet": mean_pet_dict.get("pet"),
            },
        )

        return feature

    return terra_climate_dataset.map(spatial_aggregation)

In [9]:
df_list = []

n_province = spain_provinces["province"].nunique()

for feature in tqdm(spain_provinces.iterfeatures(), total=n_province):
    res = get_monthly_data(feature).getInfo()

    df = pd.DataFrame([r["properties"] for r in res["features"]])

    df_list.append(df)

  0%|          | 0/53 [00:00<?, ?it/s]

In [10]:
data = pd.concat(df_list)

In [11]:
data.head()

,avg_pet,avg_precipitation,date,province
0,142.446771,22.252368,2000-01-01,Araba/Álava
1,350.056457,23.672784,2000-02-01,Araba/Álava
2,667.062588,50.380843,2000-03-01,Araba/Álava
3,696.993360,122.973462,2000-04-01,Araba/Álava
4,1096.485081,69.824049,2000-05-01,Araba/Álava


In [12]:
data["date"] = pd.to_datetime(data["date"])

In [13]:
data.head()

,avg_pet,avg_precipitation,date,province
0,142.446771,22.252368,2000-01-01,Araba/Álava
1,350.056457,23.672784,2000-02-01,Araba/Álava
2,667.062588,50.380843,2000-03-01,Araba/Álava
3,696.993360,122.973462,2000-04-01,Araba/Álava
4,1096.485081,69.824049,2000-05-01,Araba/Álava


In [14]:
data = data.melt(
    id_vars=["province", "date"], value_vars=["avg_precipitation", "avg_pet"]
)

In [15]:
data.head()

,province,date,variable,value
0,Araba/Álava,2000-01-01,avg_precipitation,22.252368
1,Araba/Álava,2000-02-01,avg_precipitation,23.672784
2,Araba/Álava,2000-03-01,avg_precipitation,50.380843
3,Araba/Álava,2000-04-01,avg_precipitation,122.973462
4,Araba/Álava,2000-05-01,avg_precipitation,69.824049


In [18]:
data.to_csv('../data/climate_data.csv', index=False)